# Golden Standard SM18 Parity Validation -- HCMCBXFB012 Dipole

## Purpose
Validate the Python pipeline against the SM18 legacy C++/MATLAB analyzer on
the HCMCBXFB012 dipole measurement (stair-step outer-negative cycle).

## Dataset
- **Magnet**: HCMCBXFB012-E9000006 (dipole, m=1)
- **Cycle**: stair_step_outer_negative (2024-12-04)
- **Segments**: 5 (z = -1, -0.5, 0, +0.5, +1 m)
- **Turns**: ~57,019 per segment
- **Reference options**: `dri rot nor cel dit`

## Strategy
1. Validate Kn (file vs head-CSV computation)
2. Sweep `drift_mode`, `legacy_rotate_excludes_last`, `merge_mode` on Segment 3
3. Run best configuration on all 5 segments
4. Report turn-by-turn differences (target: zero apart from numerical noise)

## How to replicate via the GUI
See the **GUI Replication Instructions** cell at the end of this notebook.

In [1]:
from pathlib import Path
import sys, gc, itertools
import numpy as np
import pandas as pd

repo_root = Path("../..").resolve()
if str(repo_root) not in sys.path:
    sys.path.insert(0, str(repo_root))

from rotating_coil_analyzer.analysis.kn_pipeline import (
    load_segment_kn_txt, compute_legacy_kn_per_turn, merge_coefficients,
)
from rotating_coil_analyzer.analysis.kn_head import (
    compute_head_kn_from_csv, compute_segment_kn_from_head,
)
from rotating_coil_analyzer.ingest.readers_streaming import StreamingReader

# ============================================================
# DATASET CONFIGURATION (from Parameters.txt)
# ============================================================
GOLDEN_DIR = Path("../../golden_standards/golden_standard_SM18_01")
BASE = "HCMCBXFB012-E9000006_20241204_150437_stair_step_outer_negative"
MEAS_DIR = GOLDEN_DIR / BASE / "aperture1"
HEAD_CSV = MEAS_DIR / "CRMMMMH_AD-00000001.csv"

MAGNET_ORDER = 1           # Dipole
R_REF_M      = 0.050       # Reference radius [m]
SAMPLES_PER_TURN = 512     # Encoder resolution
SHAFT_SPEED_RPM  = 60      # |Parameters.Measurement.v| = 60
ABS_CALIB    = 1.0         # Parameters.Measurement.absgain
N_SEGMENTS   = 5

print("Configuration:")
print(f"  Magnet order  : {MAGNET_ORDER} (dipole)")
print(f"  Rref          : {R_REF_M} m")
print(f"  Samples/turn  : {SAMPLES_PER_TURN}")
print(f"  Shaft speed   : {SHAFT_SPEED_RPM} RPM")
print(f"  Dataset       : {MEAS_DIR.resolve()}")

Configuration:
  Magnet order  : 1 (dipole)
  Rref          : 0.05 m
  Samples/turn  : 512
  Shaft speed   : 60 RPM
  Dataset       : C:\Users\albellel\python-projects\rotating-coil-analyzer\golden_standards\golden_standard_SM18_01\HCMCBXFB012-E9000006_20241204_150437_stair_step_outer_negative\aperture1


In [2]:
# ============================================================
# Parse Parameters.txt  ->  per-segment configuration
# ============================================================
BS = chr(92)  # backslash character (avoids escape issues in notebooks)

params_path = MEAS_DIR / "Parameters.txt"
params = {}
with open(params_path, "r") as f:
    for line in f:
        line = line.strip()
        if not line:
            continue
        idx = line.find(":")
        if idx < 0:
            continue
        params[line[:idx].strip()] = line[idx + 1:].strip()


def parse_table(raw):
    """Parse TABLE{r1c1\\tr1c2\\nr2c1\\tr2c2} into list-of-lists."""
    s = raw
    if s.startswith("TABLE{") and s.endswith("}"):
        s = s[6:-1]
    return [row.split(BS + "t") for row in s.split(BS + "n")]


# FDI mapping:  segment_num -> (fdi_abs_channel, fdi_cmp_channel)
fdi_rows = parse_table(params["Measurement.AP1.FDIs"])
fdi_map = {int(r[0]): (int(r[1]), int(r[2])) for r in fdi_rows}

# Connection strings:  fdi_channel -> coil connection formula
conn_rows = parse_table(params["Measurement.AP1.kn.calc.Connections"])
fdi_to_conn = {int(r[1]): r[0] for r in conn_rows}

# Build per-segment config dict
SEG_CFG = {}
for seg in range(1, N_SEGMENTS + 1):
    fa, fc = fdi_map[seg]
    SEG_CFG[seg] = dict(
        abs_conn=fdi_to_conn[fa],
        cmp_conn=fdi_to_conn[fc],
        bin_file=MEAS_DIR / f"{BASE}_corr_sigs_Ap_1_Seg{seg}.bin",
        kn_file=MEAS_DIR / f"{BASE}_Kn_values_Ap_1_Seg_{seg}.txt",
        ref_file=MEAS_DIR / f"{BASE}_results_Ap_1_Seg_{seg}.txt",
    )
    print(f"Seg {seg}: abs='{SEG_CFG[seg]['abs_conn']}'  "
          f"cmp='{SEG_CFG[seg]['cmp_conn']}'")

# Show analyzer settings from Parameters.txt
print("\nParameters.txt analyzer settings:")
for opt in ["dri", "rot", "nor", "cel", "dit", "fed"]:
    print(f"  {opt}: {params.get(f'Parameters.magnetAnalyzer.{opt}', 'N/A')}")
print(f"  magnetOrder : {params.get('Parameters.magnetAnalyzer.magnetOrder', 'N/A')}")
print(f"  refRadius   : {params.get('Parameters.magnetAnalyzer.refRadius', 'N/A')}")

Seg 1: abs='1.5'  cmp='-1.3+1.5'
Seg 2: abs='2.5'  cmp='-2.3+2.5'
Seg 3: abs='3.5'  cmp='-3.3+3.5'
Seg 4: abs='4.1'  cmp='-4.3+4.1'
Seg 5: abs='5.5'  cmp='-5.3+5.5'

Parameters.txt analyzer settings:
  dri: true
  rot: true
  nor: true
  cel: true
  dit: true
  fed: false
  magnetOrder : 1
  refRadius   : 0.050


In [3]:
# ============================================================
# Kn VALIDATION:  file Kn  vs  head-CSV computation
# ============================================================
# IMPORTANT: The reference Kn files were generated with COLD geometry.
# The scaling factors differ: warm uses ~0.999999, cold uses ~0.997/0.995.
# We must use warm_geometry=False to match.
# ============================================================
print("=" * 70)
print("Kn VALIDATION")
print("=" * 70)

# 1) Load Kn from the provided text files (one per segment)
file_kn = {}
for seg in range(1, N_SEGMENTS + 1):
    file_kn[seg] = load_segment_kn_txt(str(SEG_CFG[seg]["kn_file"]))

# 2) Compute Kn from measurement-head CSV + connection formulas
#    Use warm_geometry=False to match the legacy C++ analyzer (cold geometry).
head = compute_head_kn_from_csv(
    str(HEAD_CSV),
    warm_geometry=False,       # COLD geometry to match reference Kn files
    n_multipoles=15,
    use_design_radius=False,   # Parameters.txt: useDesignRadius=false
)
print(f"Head CSV: {HEAD_CSV.name}  ({len(head.kn_by_index)} coils, cold geometry)")

computed_kn = {}
for seg in range(1, N_SEGMENTS + 1):
    cfg = SEG_CFG[seg]
    computed_kn[seg] = compute_segment_kn_from_head(
        head,
        abs_connection=cfg["abs_conn"],
        cmp_connection=cfg["cmp_conn"],
    )

# 3) Compare all harmonics
print(f"\n{'Seg':>4} {'n':>3} {'Ch':>4} {'|file|':>14} {'|computed|':>14} {'rel_diff':>14}")
print("-" * 60)
worst_kn_diff = 0.0
for seg in range(1, N_SEGMENTS + 1):
    fk, ck = file_kn[seg], computed_kn[seg]
    for ch_name, fch, cch in [("abs", fk.kn_abs, ck.kn_abs),
                               ("cmp", fk.kn_cmp, ck.kn_cmp)]:
        for i in range(min(3, len(fk.orders))):
            fa, ca = abs(fch[i]), abs(cch[i])
            rd = abs(fa - ca) / max(fa, 1e-30) if fa > 1e-30 else 0.0
            worst_kn_diff = max(worst_kn_diff, rd)
            if i == 0:
                print(f"{seg:4d} {i+1:3d} {ch_name:>4s} {fa:14.6e} {ca:14.6e} {rd:14.6e}")

print(f"\nWorst Kn relative difference (n=1..3, abs+cmp): {worst_kn_diff:.2e}")
if worst_kn_diff < 1e-10:
    print("  --> EXCELLENT: file Kn and head-CSV agree to machine precision")
elif worst_kn_diff < 1e-6:
    print("  --> GOOD: agree within 1 ppm")
else:
    print(f"  --> WARNING: differ by {worst_kn_diff:.2e}")

Kn VALIDATION
Head CSV: CRMMMMH_AD-00000001.csv  (25 coils, cold geometry)

 Seg   n   Ch         |file|     |computed|       rel_diff
------------------------------------------------------------
   1   1  abs   1.599240e-01   1.599240e-01   8.920717e-14
   1   1  cmp   2.435875e-04   2.435875e-04   6.198607e-11
   2   1  abs   1.597376e-01   1.597376e-01   5.091089e-14
   2   1  cmp   3.194965e-05   3.194965e-05   2.423753e-10
   3   1  abs   1.598418e-01   1.598418e-01   8.369644e-14
   3   1  cmp   1.568168e-04   1.568168e-04   8.177106e-11
   4   1  abs   1.597340e-01   1.597340e-01   3.249334e-14
   4   1  cmp   2.161019e-05   2.161019e-05   2.543061e-10
   5   1  abs   1.599516e-01   1.599516e-01   1.225085e-13
   5   1  cmp   2.066188e-04   2.066188e-04   9.658491e-11

Worst Kn relative difference (n=1..3, abs+cmp): 2.54e-10
  --> GOOD: agree within 1 ppm


In [4]:
# ============================================================
# Load reference results for all 5 segments
# ============================================================
print("Loading reference results...")
ref_data = {}
for seg in range(1, N_SEGMENTS + 1):
    path = str(SEG_CFG[seg]["ref_file"])
    df = pd.read_csv(path, sep=chr(9))  # chr(9) = tab
    ref_data[seg] = df
    I = df["I(A)"].values
    print(f"  Seg {seg}: {len(df):,} turns  |  "
          f"I=[{I.min():.1f}, {I.max():.1f}] A  |  "
          f"B_main=[{df['B_main(T)'].min():.4e}, {df['B_main(T)'].max():.4e}] T")

print(f"\nReference columns ({len(ref_data[1].columns)}):")
print("  " + ", ".join(ref_data[1].columns[:12]) + ", ...")
print(f"\nOptions in reference: '{ref_data[1]['Options'].iloc[0].strip()}'")

Loading reference results...
  Seg 1: 57,019 turns  |  I=[-0.0, 1740.0] A  |  B_main=[-5.0972e-03, 2.5169e-03] T
  Seg 2: 57,019 turns  |  I=[-0.0, 1740.0] A  |  B_main=[-1.4159e+00, 1.7434e+00] T
  Seg 3: 57,019 turns  |  I=[-0.0, 1740.0] A  |  B_main=[-2.2670e+00, 2.9598e+00] T
  Seg 4: 57,019 turns  |  I=[-0.0, 1740.0] A  |  B_main=[-1.4119e+00, 1.7321e+00] T
  Seg 5: 57,019 turns  |  I=[-0.0, 1740.0] A  |  B_main=[-5.1046e-03, 2.6870e-03] T

Reference columns (46):
  Time(s), Duration(s), Options, Rref(m), Lcoil(m), I(A), Ramprate(A/s), I1(A), Ramprate1(A/s), dx(mm), dy(mm), phi(rad), ...

Options in reference: 'dri rot nor cel dit'


In [5]:
# ============================================================
# DIAGNOSTIC: Current profile, turn alignment, plateau/ramp identification
# ============================================================
# Key questions:
#   1. Which binary current column matches the reference I(A)?
#   2. Where are the plateaus and ramps?
#   3. Are the 3 missing turns at the start or end?
#   4. Does the reference include ALL turns or only plateau turns?
# ============================================================
%matplotlib widget
import matplotlib.pyplot as plt

DIAG_SEG = 3  # central segment for diagnostics

# Load binary data for diagnostic segment
reader = StreamingReader()
frame = reader.read(
    str(SEG_CFG[DIAG_SEG]["bin_file"]),
    run_id="diag", segment=str(DIAG_SEG),
    samples_per_turn=SAMPLES_PER_TURN, shaft_speed_rpm=SHAFT_SPEED_RPM,
)
Ns = SAMPLES_PER_TURN
nt_bin = frame.n_turns
nt_ref = len(ref_data[DIAG_SEG])
print(f"Binary: {nt_bin} turns | Reference: {nt_ref} turns | Delta: {nt_bin - nt_ref}")
print(f"Reader warnings: {frame.warnings}")

# Extract per-turn arrays
t_all = frame.df["t"].values.reshape(nt_bin, Ns)
t_mid = np.mean(t_all, axis=1)

# Check ALL current channels (I0, I1, etc.)
I_cols = [c for c in frame.df.columns if c.startswith("I") and c != "I"]
print(f"\nCurrent columns in binary: ['I' (main)] + {I_cols}")

I_main = frame.df["I"].values.reshape(nt_bin, Ns)
I_main_mean = np.mean(I_main, axis=1)

ref_seg = ref_data[DIAG_SEG]
ref_t = ref_seg["Time(s)"].values
ref_I = ref_seg["I(A)"].values

# Test which current column matches reference I(A)
print("\n--- CURRENT CHANNEL IDENTIFICATION ---")
for col_name in ["I"] + I_cols:
    col_data = frame.df[col_name].values.reshape(nt_bin, Ns)
    col_mean = np.mean(col_data, axis=1)
    n_cmp = min(nt_bin, nt_ref)
    dI = col_mean[:n_cmp] - ref_I[:n_cmp]
    print(f"  {col_name:4s}: dI_mean={np.mean(np.abs(dI)):10.4f} A, "
          f"dI_max={np.max(np.abs(dI)):10.4f} A, "
          f"range=[{col_mean.min():.1f}, {col_mean.max():.1f}] A")

# Time alignment check
print("\n--- TIME ALIGNMENT ---")
n_cmp = min(nt_bin, nt_ref)
dt = t_mid[:n_cmp] - ref_t[:n_cmp]
print(f"  dt_mean = {np.mean(dt):.6f} s")
print(f"  dt_std  = {np.std(dt):.6f} s")
print(f"  dt_max  = {np.max(np.abs(dt)):.6f} s")
print(f"  Binary t_start = {t_mid[0]:.6f} s, ref t_start = {ref_t[0]:.6f} s")
print(f"  Binary t_end   = {t_mid[-1]:.6f} s, ref t_end   = {ref_t[-1]:.6f} s")

# Reference time continuity check
ref_dt = np.diff(ref_t)
print(f"\n--- REFERENCE TIME CONTINUITY ---")
print(f"  ref_dt_median = {np.median(ref_dt):.6f} s")
print(f"  ref_dt_min    = {np.min(ref_dt):.6f} s")
print(f"  ref_dt_max    = {np.max(ref_dt):.6f} s")
gaps = np.where(ref_dt > 1.5 * np.median(ref_dt))[0]
if len(gaps) > 0:
    print(f"  TIME GAPS at turns: {gaps[:20]} (potential missing turns)")
else:
    print(f"  No time gaps -> ALL turns present in reference (no filtering)")

# Identify plateaus and ramps from reference Ramprate
if "Ramprate(A/s)" in ref_seg.columns:
    rr = ref_seg["Ramprate(A/s)"].values.astype(float)
    plateau_mask = np.abs(rr) < 0.5  # effectively zero ramprate
    n_plateau = int(np.sum(plateau_mask))
    n_ramp = int(np.sum(~plateau_mask))
    print(f"\n--- PLATEAU vs RAMP (from reference Ramprate column) ---")
    print(f"  Plateau turns (|ramprate| < 0.5 A/s): {n_plateau:,}")
    print(f"  Ramp turns (|ramprate| >= 0.5 A/s):   {n_ramp:,}")
    print(f"  Fraction plateau: {n_plateau/len(rr)*100:.1f}%")
else:
    print("\n  No Ramprate column in reference -- using current derivative instead")
    ref_dI = np.diff(ref_I)
    plateau_mask = np.zeros(len(ref_I), dtype=bool)
    plateau_mask[1:] = np.abs(ref_dI) < 0.5
    plateau_mask[0] = plateau_mask[1]
    n_plateau = int(np.sum(plateau_mask))
    n_ramp = int(np.sum(~plateau_mask))
    print(f"  Plateau turns: {n_plateau:,} | Ramp turns: {n_ramp:,}")

# Current profile overview
fig, axes = plt.subplots(2, 2, figsize=(16, 8))

ax = axes[0, 0]
ax.plot(ref_t / 60, ref_I, linewidth=0.3, color="blue")
ax.set_xlabel("Time (min)")
ax.set_ylabel("I (A)")
ax.set_title("Reference current profile (stair-step)")
ax.grid(True, alpha=0.3)

ax = axes[0, 1]
ax.plot(ref_t[:6000] / 60, ref_I[:6000], linewidth=0.5, color="blue")
ax.set_xlabel("Time (min)")
ax.set_ylabel("I (A)")
ax.set_title("First 6000 turns (zoom)")
ax.grid(True, alpha=0.3)

ax = axes[1, 0]
if "Ramprate(A/s)" in ref_seg.columns:
    ax.plot(ref_t / 60, rr, linewidth=0.2, color="red")
    ax.set_ylabel("Ramprate (A/s)")
else:
    ax.plot(ref_t[1:] / 60, np.diff(ref_I), linewidth=0.2, color="red")
    ax.set_ylabel("dI/dt (A/turn)")
ax.set_xlabel("Time (min)")
ax.set_title("Current ramp rate")
ax.grid(True, alpha=0.3)

ax = axes[1, 1]
ax.scatter(range(len(plateau_mask)), plateau_mask.astype(float),
           s=0.1, alpha=0.3, c=np.where(plateau_mask, "green", "red"))
ax.set_xlabel("Turn")
ax.set_ylabel("Is plateau")
ax.set_title("Plateau (green) vs Ramp (red) turns")
ax.set_yticks([0, 1])
ax.set_yticklabels(["Ramp", "Plateau"])

plt.tight_layout()
plt.savefig("golden_standard_SM18_diagnostics.png", dpi=150)
plt.show()

# Store plateau mask for later use
PLATEAU_MASK = plateau_mask

del frame
gc.collect()
print("\nDiagnostics complete.")

Binary: 57022 turns | Reference: 57019 turns | Delta: 3
Reader warnings: ('dt nominal check: dt_med=0.001939, dt_nom=0.00195312, rel_err=0.00723', 'Selected binary format: float64_le_5col', 'column map: t=0, df_abs=1, df_cmp=2, current cols=3..4', 'flux ranges (p99.5-p0.5): abs=0.0112473, cmp=1.14923e-05', 'current candidate ranges (p99.5-p0.5): col4:2340.46, col3:1740.04')

Current columns in binary: ['I' (main)] + ['I0', 'I1']

--- CURRENT CHANNEL IDENTIFICATION ---
  I   : dI_mean=   62.7248 A, dI_max= 1430.0053 A, range=[-1430.0, 1135.0] A
  I0  : dI_mean=    0.0000 A, dI_max=    0.0000 A, range=[-0.0, 1740.0] A
  I1  : dI_mean=   62.7248 A, dI_max= 1430.0053 A, range=[-1430.0, 1135.0] A

--- TIME ALIGNMENT ---
  dt_mean = -0.007632 s
  dt_std  = 0.000711 s
  dt_max  = 0.009478 s
  Binary t_start = 0.507149 s, ref t_start = 0.512632 s
  Binary t_end   = 57648.220709 s, ref t_end   = 57645.194751 s

--- REFERENCE TIME CONTINUITY ---
  ref_dt_median = 1.010919 s
  ref_dt_min    = 1.0

C:\Users\albellel\AppData\Local\Temp\ipykernel_23964\1500203291.py:136: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


In [6]:
# ============================================================
# Helper:  load binary, run pipeline, compare with reference
# ============================================================

def load_segment_arrays(seg_num, max_turns=None):
    """Read binary file -> turn-reshaped arrays."""
    cfg = SEG_CFG[seg_num]
    reader = StreamingReader()
    frame = reader.read(
        str(cfg["bin_file"]),
        run_id="golden",
        segment=str(seg_num),
        samples_per_turn=SAMPLES_PER_TURN,
        shaft_speed_rpm=SHAFT_SPEED_RPM,
    )
    Ns = SAMPLES_PER_TURN
    nt = frame.n_turns
    if max_turns is not None:
        nt = min(nt, max_turns)
    n = nt * Ns
    arrays = dict(
        df_abs=frame.df["df_abs"].values[:n].reshape(nt, Ns).copy(),
        df_cmp=frame.df["df_cmp"].values[:n].reshape(nt, Ns).copy(),
        t=frame.df["t"].values[:n].reshape(nt, Ns).copy(),
        I=frame.df["I"].values[:n].reshape(nt, Ns).copy(),
        n_turns=nt,
        warnings=frame.warnings,
    )
    del frame
    gc.collect()
    return arrays


def run_pipeline(arrays, kn, *, options, drift_mode, legacy_rotate_excludes_last):
    """Run compute_legacy_kn_per_turn on pre-loaded arrays."""
    return compute_legacy_kn_per_turn(
        df_abs_turns=arrays["df_abs"],
        df_cmp_turns=arrays["df_cmp"],
        t_turns=arrays["t"],
        I_turns=arrays["I"],
        kn=kn,
        Rref_m=R_REF_M,
        magnet_order=MAGNET_ORDER,
        absCalib=ABS_CALIB,
        options=options,
        drift_mode=drift_mode,
        legacy_rotate_excludes_last=legacy_rotate_excludes_last,
    )


def compare_with_ref(result, C_merged, ref_df, n_turns):
    """Compare pipeline output with reference -> metrics dict.

    Clips to the minimum of n_turns and len(ref_df) to handle
    slight turn-count differences between binary reader and reference file.
    """
    n = min(n_turns, len(ref_df))
    ref = ref_df.iloc[:n]
    m = {"n_compare": n}

    # B_main(T)
    comp_Bm = result.main_field.real[:n]
    ref_Bm = ref["B_main(T)"].values.astype(float)
    d = comp_Bm - ref_Bm
    m["Bmain_maxabs"] = float(np.max(np.abs(d)))
    m["Bmain_rms"] = float(np.sqrt(np.mean(d ** 2)))

    # phi(rad)
    d_phi = result.phi_out_rad[:n] - ref["phi(rad)"].values.astype(float)
    m["phi_maxabs"] = float(np.max(np.abs(d_phi)))

    # dx(mm), dy(mm)
    d_dx = result.x_m[:n] * 1000.0 - ref["dx(mm)"].values.astype(float)
    d_dy = result.y_m[:n] * 1000.0 - ref["dy(mm)"].values.astype(float)
    m["dx_maxabs"] = float(np.max(np.abs(d_dx)))
    m["dy_maxabs"] = float(np.max(np.abs(d_dy)))

    # Per-harmonic: B1(T), b2(Units)..b15(Units), A1(T), a2..a15
    for h in range(1, 16):
        comp_b = C_merged[:n, h - 1].real
        comp_a = C_merged[:n, h - 1].imag
        if h == 1:
            bcol, acol = "B1(T)", "A1(T)"
        else:
            bcol, acol = f"b{h}(Units)", f"a{h}(Units)"
        if bcol in ref.columns:
            db = comp_b - ref[bcol].values.astype(float)
            m[f"b{h}_maxabs"] = float(np.max(np.abs(db)))
            m[f"b{h}_rms"] = float(np.sqrt(np.mean(db ** 2)))
        if acol in ref.columns:
            da = comp_a - ref[acol].values.astype(float)
            m[f"a{h}_maxabs"] = float(np.max(np.abs(da)))
            m[f"a{h}_rms"] = float(np.sqrt(np.mean(da ** 2)))

    # Aggregate score = sum of all bn maxabs
    m["total_score"] = sum(m.get(f"b{h}_maxabs", 0) for h in range(1, 16))
    return m


print("Helper functions defined.")

Helper functions defined.


In [7]:
# ============================================================
# PARAMETER SWEEP on Segment 3
# ============================================================
# Use ALL turns for the sweep (not just first 2000).
# After sweep, report metrics separately for plateau and ramp turns.
# ============================================================

SWEEP_SEG = 3
SWEEP_MAX_TURNS = None  # use ALL turns

print(f"Loading Segment {SWEEP_SEG} data (ALL turns)...")
sweep_arrays = load_segment_arrays(SWEEP_SEG, max_turns=SWEEP_MAX_TURNS)
nt_sweep = sweep_arrays["n_turns"]
nt_ref_sweep = len(ref_data[SWEEP_SEG])
nt_cmp = min(nt_sweep, nt_ref_sweep)
print(f"  Loaded {nt_sweep} turns, comparing {nt_cmp} turns")

# Define search grid
option_sets = {
    "dri+rot+nor+cel+dit":      ("dri", "rot", "nor", "cel", "dit"),
    "dri+rot+nor+cel+dit+fed":  ("dri", "rot", "nor", "cel", "dit", "fed"),
    "dri+rot+nor+dit":          ("dri", "rot", "nor", "dit"),
    "dri+rot+nor+cel":          ("dri", "rot", "nor", "cel"),
    "dri+rot+cel+dit":          ("dri", "rot", "cel", "dit"),
}
drift_modes = ["legacy", "weighted"]
rot_excl_opts = [True, False]
merge_modes = ["abs_all", "abs_upto_m_cmp_above"]

sweep_results = []
kn_seg = file_kn[SWEEP_SEG]
n_combos = len(option_sets) * len(drift_modes) * len(rot_excl_opts) * len(merge_modes)
print(f"\nRunning {n_combos} parameter combinations...")

combo_idx = 0
for opt_label, opts in option_sets.items():
    for dm in drift_modes:
        for rel in rot_excl_opts:
            result = run_pipeline(
                sweep_arrays, kn_seg,
                options=opts, drift_mode=dm,
                legacy_rotate_excludes_last=rel,
            )
            for mm in merge_modes:
                combo_idx += 1
                C_merged, _ = merge_coefficients(
                    C_abs=result.C_abs, C_cmp=result.C_cmp,
                    magnet_order=MAGNET_ORDER, mode=mm,
                )
                if "nor" not in set(opts):
                    main_val = C_merged[:, MAGNET_ORDER - 1].real.copy()
                    scale = np.where(
                        np.abs(main_val) > 1e-20,
                        10000.0 / main_val, np.nan,
                    )
                    C_merged = C_merged * scale[:, None]

                # Overall metrics
                metrics = compare_with_ref(
                    result, C_merged,
                    ref_data[SWEEP_SEG], nt_sweep,
                )
                metrics["options"] = opt_label
                metrics["drift_mode"] = dm
                metrics["rot_excl_last"] = rel
                metrics["merge_mode"] = mm

                # PLATEAU-ONLY metrics (where ramprate ~ 0)
                pmask = PLATEAU_MASK[:nt_cmp]
                if pmask.sum() > 100:
                    ref_plat = ref_data[SWEEP_SEG].iloc[:nt_cmp][pmask]
                    n_plat = len(ref_plat)
                    comp_Bm_plat = result.main_field.real[:nt_cmp][pmask]
                    ref_Bm_plat = ref_plat["B_main(T)"].values.astype(float)
                    d_plat = comp_Bm_plat - ref_Bm_plat
                    metrics["plat_Bmain_maxabs"] = float(np.max(np.abs(d_plat)))
                    # Plateau b3
                    C_plat = C_merged[:nt_cmp][pmask]
                    if "b3(Units)" in ref_plat.columns:
                        db3_plat = C_plat[:, 2].real - ref_plat["b3(Units)"].values.astype(float)
                        metrics["plat_b3_maxabs"] = float(np.max(np.abs(db3_plat)))
                    # Plateau total score (sum bn maxabs over plateau turns)
                    plat_score = 0.0
                    for h in range(1, 16):
                        bcol = "B1(T)" if h == 1 else f"b{h}(Units)"
                        if bcol in ref_plat.columns:
                            db = C_plat[:, h-1].real - ref_plat[bcol].values.astype(float)
                            plat_score += float(np.max(np.abs(db)))
                    metrics["plat_total_score"] = plat_score
                    metrics["n_plateau"] = n_plat

                sweep_results.append(metrics)

            del result
            gc.collect()

sweep_df = pd.DataFrame(sweep_results)
print(f"Sweep complete: {len(sweep_df)} combinations evaluated.")

# Show top 10 by total_score (ALL turns)
sweep_df_sorted = sweep_df.sort_values("total_score")
print("\n" + "=" * 120)
print("TOP 10 CONFIGURATIONS -- ALL TURNS (by total harmonic score)")
print("=" * 120)
show_cols = ["options", "drift_mode", "rot_excl_last", "merge_mode",
             "total_score", "Bmain_maxabs", "phi_maxabs", "b1_maxabs"]
print(sweep_df_sorted[show_cols].head(10).to_string(index=False))

# Show top 10 by PLATEAU score
if "plat_total_score" in sweep_df.columns:
    sweep_plat_sorted = sweep_df.sort_values("plat_total_score")
    print("\n" + "=" * 120)
    print("TOP 10 CONFIGURATIONS -- PLATEAU TURNS ONLY")
    print("=" * 120)
    pcols = ["options", "drift_mode", "rot_excl_last", "merge_mode",
             "plat_total_score", "plat_Bmain_maxabs", "n_plateau"]
    pcols = [c for c in pcols if c in sweep_df.columns]
    print(sweep_plat_sorted[pcols].head(10).to_string(index=False))

    best_plat = sweep_plat_sorted.iloc[0]
    print(f"\n{'='*70}")
    print("BEST CONFIGURATION (PLATEAU TURNS):")
    print(f"  options              : {best_plat['options']}")
    print(f"  drift_mode           : {best_plat['drift_mode']}")
    print(f"  rot_excl_last        : {best_plat['rot_excl_last']}")
    print(f"  merge_mode           : {best_plat['merge_mode']}")
    print(f"  plat_total_score     : {best_plat['plat_total_score']:.6e}")
    print(f"  plat_Bmain max|diff| : {best_plat['plat_Bmain_maxabs']:.6e}")
    print(f"  n_plateau            : {best_plat.get('n_plateau', 'N/A')}")
    print(f"{'='*70}")

best = sweep_df_sorted.iloc[0]
print(f"\n{'='*70}")
print("BEST CONFIGURATION (ALL TURNS):")
print(f"  options              : {best['options']}")
print(f"  drift_mode           : {best['drift_mode']}")
print(f"  rot_excl_last        : {best['rot_excl_last']}")
print(f"  merge_mode           : {best['merge_mode']}")
print(f"  total_score          : {best['total_score']:.6e}")
print(f"  Bmain max|diff|      : {best['Bmain_maxabs']:.6e}")
print(f"  phi max|diff|        : {best['phi_maxabs']:.6e}")
print(f"{'='*70}")

del sweep_arrays
gc.collect()

Loading Segment 3 data (ALL turns)...
  Loaded 57022 turns, comparing 57019 turns

Running 40 parameter combinations...
Sweep complete: 40 combinations evaluated.

TOP 10 CONFIGURATIONS -- ALL TURNS (by total harmonic score)
            options drift_mode  rot_excl_last           merge_mode  total_score  Bmain_maxabs  phi_maxabs    b1_maxabs
    dri+rot+nor+cel     legacy           True abs_upto_m_cmp_above   135.219444      0.000523    0.017620 1.818989e-12
dri+rot+nor+cel+dit     legacy           True abs_upto_m_cmp_above   182.343020      4.531945    0.199160 1.818989e-12
    dri+rot+nor+dit     legacy           True abs_upto_m_cmp_above   182.343020      4.531945    0.199160 1.818989e-12
    dri+rot+cel+dit     legacy           True abs_upto_m_cmp_above   182.343020      4.531945    0.199160 1.818989e-12
    dri+rot+nor+cel   weighted           True abs_upto_m_cmp_above   338.551177      0.002176    0.025033 1.818989e-12
    dri+rot+nor+dit   weighted           True abs_upto_m_cmp_

0

In [8]:
# ============================================================
# FULL VALIDATION with best config on ALL 5 SEGMENTS
# ============================================================
# Report both ALL-TURNS and PLATEAU-ONLY metrics.

BEST_OPTIONS_LABEL = best["options"]
BEST_OPTIONS = dict(option_sets)[BEST_OPTIONS_LABEL]
BEST_DRIFT = best["drift_mode"]
BEST_ROT_EXCL = bool(best["rot_excl_last"])
BEST_MERGE = best["merge_mode"]
BEST_NOR_IN_OPTS = "nor" in set(BEST_OPTIONS)

print("Running full validation with best configuration:")
print(f"  options  : {BEST_OPTIONS}")
print(f"  drift    : {BEST_DRIFT}")
print(f"  rot_excl : {BEST_ROT_EXCL}")
print(f"  merge    : {BEST_MERGE}")
print()

full_metrics = {}
full_plat_metrics = {}
full_arrays_cache = {}

for seg in range(1, N_SEGMENTS + 1):
    print(f"--- Segment {seg} ---")
    arrays = load_segment_arrays(seg)
    nt_bin = arrays["n_turns"]
    nt_ref = len(ref_data[seg])
    nt = min(nt_bin, nt_ref)
    print(f"  Binary: {nt_bin:,} | Ref: {nt_ref:,} | Comparing: {nt:,}")

    result = run_pipeline(
        arrays, file_kn[seg],
        options=BEST_OPTIONS, drift_mode=BEST_DRIFT,
        legacy_rotate_excludes_last=BEST_ROT_EXCL,
    )
    C_merged, choice = merge_coefficients(
        C_abs=result.C_abs, C_cmp=result.C_cmp,
        magnet_order=MAGNET_ORDER, mode=BEST_MERGE,
    )
    if not BEST_NOR_IN_OPTS:
        main_val = C_merged[:, MAGNET_ORDER - 1].real.copy()
        scale = np.where(np.abs(main_val) > 1e-20, 10000.0 / main_val, np.nan)
        C_merged = C_merged * scale[:, None]

    # ALL-turns metrics
    metrics = compare_with_ref(result, C_merged, ref_data[seg], nt_bin)
    full_metrics[seg] = metrics

    # PLATEAU-ONLY metrics (build per-segment plateau mask from ramprate)
    ref_seg = ref_data[seg]
    if "Ramprate(A/s)" in ref_seg.columns:
        rr_seg = ref_seg["Ramprate(A/s)"].values.astype(float)
        pmask_seg = np.abs(rr_seg) < 0.5
    else:
        ref_dI = np.diff(ref_seg["I(A)"].values.astype(float))
        pmask_seg = np.zeros(len(ref_seg), dtype=bool)
        pmask_seg[1:] = np.abs(ref_dI) < 0.5
        pmask_seg[0] = pmask_seg[1]

    n_plat = int(pmask_seg[:nt].sum())
    plat_m = {}
    if n_plat > 100:
        ref_plat = ref_seg.iloc[:nt][pmask_seg[:nt]]
        comp_Bm_plat = result.main_field.real[:nt][pmask_seg[:nt]]
        ref_Bm_plat = ref_plat["B_main(T)"].values.astype(float)
        d_Bm_plat = comp_Bm_plat - ref_Bm_plat
        plat_m["Bmain_maxabs"] = float(np.max(np.abs(d_Bm_plat)))
        plat_m["Bmain_rms"] = float(np.sqrt(np.mean(d_Bm_plat ** 2)))

        C_plat = C_merged[:nt][pmask_seg[:nt]]
        plat_score = 0.0
        for h in range(1, 16):
            bcol = "B1(T)" if h == 1 else f"b{h}(Units)"
            if bcol in ref_plat.columns:
                db = C_plat[:, h-1].real - ref_plat[bcol].values.astype(float)
                plat_m[f"b{h}_maxabs"] = float(np.max(np.abs(db)))
                plat_m[f"b{h}_rms"] = float(np.sqrt(np.mean(db ** 2)))
                plat_score += plat_m[f"b{h}_maxabs"]
        plat_m["total_score"] = plat_score
        plat_m["n_plateau"] = n_plat

    full_plat_metrics[seg] = plat_m

    full_arrays_cache[seg] = dict(
        result=result, C_merged=C_merged, n_turns=nt, choice=choice,
        plateau_mask=pmask_seg[:nt],
    )

    # Print summary (both all-turns and plateau)
    print(f"  ALL TURNS:")
    print(f"    B_main max|diff| = {metrics['Bmain_maxabs']:.6e} T")
    print(f"    phi    max|diff| = {metrics['phi_maxabs']:.6e} rad")
    print(f"    Total score      = {metrics['total_score']:.6e}")
    if plat_m:
        print(f"  PLATEAU ONLY ({n_plat:,} turns):")
        print(f"    B_main max|diff| = {plat_m['Bmain_maxabs']:.6e} T")
        print(f"    Total score      = {plat_m['total_score']:.6e}")
        for h in [1, 3, 5]:
            key = f"b{h}_maxabs"
            if key in plat_m:
                lbl = "B1(T)" if h == 1 else f"b{h}(U)"
                print(f"    {lbl:>8s} max|diff| = {plat_m[key]:.6e}")
    print()

    del arrays
    gc.collect()

# Aggregate summary
print("=" * 100)
print("AGGREGATE SUMMARY")
print("=" * 100)
print(f"\n{'Seg':>4} {'ALL total':>14} {'ALL Bmain':>14} {'PLAT total':>14} "
      f"{'PLAT Bmain':>14} {'n_plat':>10}")
print("-" * 80)
for seg in range(1, N_SEGMENTS + 1):
    m = full_metrics[seg]
    pm = full_plat_metrics[seg]
    pt = pm.get("total_score", float("nan"))
    pb = pm.get("Bmain_maxabs", float("nan"))
    np_ = pm.get("n_plateau", 0)
    print(f"{seg:4d} {m['total_score']:14.6e} {m['Bmain_maxabs']:14.6e} "
          f"{pt:14.6e} {pb:14.6e} {np_:10d}")

Running full validation with best configuration:
  options  : ('dri', 'rot', 'nor', 'cel')
  drift    : legacy
  rot_excl : True
  merge    : abs_upto_m_cmp_above

--- Segment 1 ---
  Binary: 57,022 | Ref: 57,019 | Comparing: 57,019
  ALL TURNS:
    B_main max|diff| = 4.002155e-06 T
    phi    max|diff| = 2.301232e-01 rad
    Total score      = 4.651943e+04
  PLATEAU ONLY (56,670 turns):
    B_main max|diff| = 8.068388e-08 T
    Total score      = 4.043843e-02
       B1(T) max|diff| = 1.818989e-12
       b3(U) max|diff| = 2.317162e-03
       b5(U) max|diff| = 1.699558e-03

--- Segment 2 ---
  Binary: 57,022 | Ref: 57,019 | Comparing: 57,019
  ALL TURNS:
    B_main max|diff| = 2.852638e-04 T
    phi    max|diff| = 1.873588e-02 rad
    Total score      = 1.285834e+02
  PLATEAU ONLY (56,670 turns):
    B_main max|diff| = 1.308079e-05 T
    Total score      = 2.764359e-03
       B1(T) max|diff| = 1.818989e-12
       b3(U) max|diff| = 3.285806e-04
       b5(U) max|diff| = 2.700523e-04

--- 

In [9]:
# ============================================================
# DETAILED TURN-BY-TURN ANALYSIS + PLOTS
# ============================================================

fig, axes = plt.subplots(3, 2, figsize=(16, 14))

for seg in range(1, N_SEGMENTS + 1):
    cache = full_arrays_cache[seg]
    result = cache["result"]
    C_merged = cache["C_merged"]
    nt = cache["n_turns"]
    ref = ref_data[seg].iloc[:nt]

    # --- B_main difference ---
    diff_Bm = result.main_field.real[:nt] - ref["B_main(T)"].values.astype(float)
    axes[0, 0].plot(diff_Bm, label=f"Seg {seg}", alpha=0.5, linewidth=0.3)

    # --- b3 difference ---
    if "b3(Units)" in ref.columns:
        diff_b3 = C_merged[:nt, 2].real - ref["b3(Units)"].values.astype(float)
        axes[0, 1].plot(diff_b3, label=f"Seg {seg}", alpha=0.5, linewidth=0.3)

# Decorate top row
for ax, title, ylabel in [
    (axes[0, 0], "B_main(T) difference (all turns)", "Diff (T)"),
    (axes[0, 1], "b3(Units) difference (all turns)", "Diff (units)"),
]:
    ax.set_title(title); ax.set_ylabel(ylabel); ax.set_xlabel("Turn")
    ax.legend(fontsize=7); ax.grid(True, alpha=0.3)

# Per-turn B_main difference for Seg 3 with plateau/ramp coloring
seg3c = full_arrays_cache[3]
nt3 = seg3c["n_turns"]
ref3 = ref_data[3].iloc[:nt3]
diff_Bm3 = seg3c["result"].main_field.real[:nt3] - ref3["B_main(T)"].values.astype(float)
pmask3 = seg3c["plateau_mask"]

ax = axes[1, 0]
ax.scatter(np.where(pmask3)[0], diff_Bm3[pmask3],
           s=0.1, alpha=0.3, color="green", label="Plateau")
ax.scatter(np.where(~pmask3)[0], diff_Bm3[~pmask3],
           s=0.1, alpha=0.3, color="red", label="Ramp")
ax.set_title("Seg 3: B_main diff -- plateau (green) vs ramp (red)")
ax.set_xlabel("Turn"); ax.set_ylabel("Diff (T)")
ax.legend(fontsize=7, markerscale=20); ax.grid(True, alpha=0.3)

# b3 difference with plateau/ramp coloring
if "b3(Units)" in ref3.columns:
    diff_b33 = seg3c["C_merged"][:nt3, 2].real - ref3["b3(Units)"].values.astype(float)
    ax = axes[1, 1]
    ax.scatter(np.where(pmask3)[0], diff_b33[pmask3],
               s=0.1, alpha=0.3, color="green", label="Plateau")
    ax.scatter(np.where(~pmask3)[0], diff_b33[~pmask3],
               s=0.1, alpha=0.3, color="red", label="Ramp")
    ax.set_title("Seg 3: b3(Units) diff -- plateau vs ramp")
    ax.set_xlabel("Turn"); ax.set_ylabel("Diff (units)")
    ax.legend(fontsize=7, markerscale=20); ax.grid(True, alpha=0.3)

# Per-harmonic RMS: ALL vs PLATEAU (Segment 3)
ax = axes[2, 0]
m3_all = full_metrics[3]
m3_plat = full_plat_metrics[3]
ns = list(range(1, 16))
rms_all = [m3_all.get(f"b{n}_rms", 0) for n in ns]
rms_plat = [m3_plat.get(f"b{n}_rms", 0) for n in ns]
x = np.array(ns)
ax.bar(x - 0.15, rms_all, 0.3, label="All turns", color="coral", edgecolor="black")
ax.bar(x + 0.15, rms_plat, 0.3, label="Plateau only", color="teal", edgecolor="black")
ax.set_xlabel("Harmonic order n"); ax.set_ylabel("RMS difference")
ax.set_title("Per-harmonic RMS: All vs Plateau (Seg 3)")
ax.set_yscale("log"); ax.legend(); ax.grid(True, alpha=0.3)

# Per-segment score comparison (all vs plateau)
ax = axes[2, 1]
segs = list(range(1, N_SEGMENTS + 1))
scores_all = [full_metrics[s]["total_score"] for s in segs]
scores_plat = [full_plat_metrics[s].get("total_score", 0) for s in segs]
x = np.array(segs)
ax.bar(x - 0.15, scores_all, 0.3, label="All turns", color="coral", edgecolor="black")
ax.bar(x + 0.15, scores_plat, 0.3, label="Plateau only", color="teal", edgecolor="black")
ax.set_xlabel("Segment"); ax.set_ylabel("Total harmonic score")
ax.set_title("Total score: All vs Plateau (lower = better)")
ax.legend(); ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig("golden_standard_SM18_parity_plots.png", dpi=150)
plt.show()

# Average-level comparison for Segment 3 (plateau only)
print("\nSEGMENT 3 -- AVERAGE harmonic comparison (PLATEAU turns only):")
print(f"{'Harmonic':>10} {'Computed':>16} {'Reference':>16} {'Diff':>16} {'Rel diff':>14}")
print("-" * 75)
seg3_ref_plat = ref_data[3].iloc[:nt3][pmask3]
seg3_C_plat = seg3c["C_merged"][:nt3][pmask3]
for n in range(1, 16):
    comp_mean = np.mean(seg3_C_plat[:, n-1].real)
    bcol = "B1(T)" if n == 1 else f"b{n}(Units)"
    if bcol in seg3_ref_plat.columns:
        ref_mean = np.mean(seg3_ref_plat[bcol].values.astype(float))
        diff = comp_mean - ref_mean
        rel = abs(diff) / max(abs(ref_mean), 1e-30)
        lbl = "B1" if n == 1 else f"b{n}"
        print(f"{lbl:>10s} {comp_mean:16.8e} {ref_mean:16.8e} {diff:16.8e} {rel:14.6e}")


SEGMENT 3 -- AVERAGE harmonic comparison (PLATEAU turns only):
  Harmonic         Computed        Reference             Diff       Rel diff
---------------------------------------------------------------------------
        B1   1.00000000e+04   1.00000000e+04   0.00000000e+00   0.000000e+00
        b2  -3.10686468e+02  -3.10686468e+02   7.58461738e-10   2.441245e-12
        b3   1.32200740e+03   1.32200740e+03  -2.13185558e-09   1.612590e-12
        b4   3.03986244e+02   3.03986244e+02   2.19415597e-10   7.217945e-13
        b5  -2.40827908e+03  -2.40827908e+03  -1.76032700e-09   7.309481e-13
        b6  -4.91555016e+01  -4.91555016e+01  -2.02640393e-09   4.122436e-11
        b7   2.58763247e+02   2.58763247e+02   1.70388148e-09   6.584712e-12
        b8  -1.50683646e+01  -1.50683646e+01   1.19880994e-10   7.955807e-12
        b9   3.49928634e+02   3.49928634e+02   4.14178203e-09   1.183608e-11
       b10  -2.72804241e+01  -2.72804241e+01   1.17570664e-09   4.309708e-11
       b11   

C:\Users\albellel\AppData\Local\Temp\ipykernel_23964\102763596.py:87: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


In [10]:
# ============================================================
# PER-PLATEAU HARMONIC PRECISION ANALYSIS
# ============================================================
# For each segment, identify individual current plateaus in the
# stair-step profile, compute per-harmonic average precision at
# each plateau, and express in ppb relative to the main field.
#
# ppb convention:
#   B_main (Tesla):  ppb = |delta<Bm>| / |<Bm_ref>| * 1e9
#   bn (Units, n>=2): ppb = |delta<bn>| * 1e5
#       (1 Unit = 1e-4 of main -> ppb = diff_units * 1e-4 * 1e9)
# ============================================================

%matplotlib widget
import matplotlib.pyplot as plt

def identify_plateaus(I_vec, min_turns=50, current_round=10.0):
    """Identify individual plateaus by rounding current to nearest
    `current_round` and finding contiguous runs of the same level.
    This correctly resolves individual steps in a stair-step cycle,
    unlike ramprate-based methods which may miss gradual transitions.
    """
    n = len(I_vec)
    I_r = np.round(np.asarray(I_vec, dtype=float) / current_round) * current_round

    plateaus = []
    start = 0
    for i in range(1, n):
        if I_r[i] != I_r[start]:
            if i - start >= min_turns:
                mask = np.zeros(n, dtype=bool); mask[start:i] = True
                plateaus.append({
                    'idx': len(plateaus),
                    'I_mean': float(np.mean(I_vec[start:i])),
                    'I_level': float(I_r[start]),
                    'start': start, 'end': i,
                    'n_turns': i - start, 'mask': mask,
                })
            start = i
    if n - start >= min_turns:
        mask = np.zeros(n, dtype=bool); mask[start:n] = True
        plateaus.append({
            'idx': len(plateaus),
            'I_mean': float(np.mean(I_vec[start:n])),
            'I_level': float(I_r[start]),
            'start': start, 'end': n,
            'n_turns': n - start, 'mask': mask,
        })
    return plateaus

# ---- Compute precision for each segment ----
all_plat = {}   # seg -> list of plateau dicts
all_prec = {}   # seg -> list of precision dicts

for seg in range(1, N_SEGMENTS + 1):
    cache = full_arrays_cache[seg]
    nt = cache["n_turns"]
    C = cache["C_merged"][:nt]
    Bm = cache["result"].main_field.real[:nt]
    ref = ref_data[seg].iloc[:nt]

    I_ref = ref["I(A)"].values.astype(float)
    plateaus = identify_plateaus(I_ref, min_turns=50, current_round=10.0)
    all_plat[seg] = plateaus

    rows = []
    for p in plateaus:
        m = p['mask']
        r = {'plat': p['idx'], 'I_A': p['I_mean'], 'I_level': p['I_level'],
             'N': p['n_turns']}

        # B_main (Tesla)
        cB = Bm[m]; rB = ref["B_main(T)"].values[:nt].astype(float)[m]
        avg_c, avg_r = float(np.mean(cB)), float(np.mean(rB))
        denom = max(abs(avg_r), 1e-20)
        r['Bm_ppb'] = abs(avg_c - avg_r) / denom * 1e9
        r['Bm_maxdiff'] = float(np.max(np.abs(cB - rB)))
        r['Bm_avg_r'] = avg_r

        # bn and an (n=2..15, Units)
        Cp = C[m]
        for h in range(2, 16):
            for kind, attr in [('b', 'real'), ('a', 'imag')]:
                col = f"{kind}{h}(Units)"
                if col not in ref.columns:
                    continue
                cv = getattr(Cp[:, h-1], attr)
                rv = ref[col].values[:nt].astype(float)[m]
                avg_d = float(np.mean(cv)) - float(np.mean(rv))
                r[f'{kind}{h}_ppb'] = abs(avg_d) * 1e5
                r[f'{kind}{h}_maxdiff'] = float(np.max(np.abs(cv - rv)))
                r[f'{kind}{h}_avg_c'] = float(np.mean(cv))
                r[f'{kind}{h}_avg_r'] = float(np.mean(rv))
        rows.append(r)
    all_prec[seg] = rows
    print(f"Seg {seg}: {len(plateaus)} plateaus identified "
          f"(I levels: {[f'{p['I_level']:.0f}' for p in plateaus]})")

# ============= OVERVIEW TABLES =============
for seg in range(1, N_SEGMENTS + 1):
    rows = all_prec[seg]
    print(f"\n{'='*160}")
    print(f"SEGMENT {seg} -- PER-PLATEAU AVG PRECISION (ppb rel. main field)  "
          f"[{len(rows)} plateaus]")
    print(f"{'='*160}")
    hdr = f"{'#':>3} {'I(A)':>8} {'N':>6} {'Bmain':>10}"
    for h in range(2, 16): hdr += f" {'b'+str(h):>8}"
    hdr += f" {'worst':>8}"
    print(hdr); print("-" * len(hdr))

    for r in rows:
        w = 0
        line = f"{r['plat']:3d} {r['I_A']:8.1f} {r['N']:6d} {r['Bm_ppb']:10.4f}"
        for h in range(2, 16):
            v = r.get(f'b{h}_ppb', float('nan'))
            line += f" {v:8.4f}"
            if np.isfinite(v): w = max(w, v)
        if abs(r['I_A']) > 10: w = max(w, r['Bm_ppb'])
        line += f" {w:8.4f}"
        print(line)

# ============= b3 DETAIL TABLE =============
print(f"\n{'='*130}")
print("b3 COMPONENT -- ALL SEGMENTS, ALL PLATEAUS")
print(f"{'='*130}")
print(f"{'Seg':>4} {'#':>3} {'I(A)':>8} {'N':>6} {'b3_comp':>14} {'b3_ref':>14} "
      f"{'diff':>12} {'ppb':>10} {'max|diff|':>12}")
print("-" * 110)
for seg in range(1, N_SEGMENTS + 1):
    for r in all_prec[seg]:
        bc = r.get('b3_avg_c', np.nan)
        br = r.get('b3_avg_r', np.nan)
        bd = bc - br if np.isfinite(bc) and np.isfinite(br) else np.nan
        print(f"{seg:4d} {r['plat']:3d} {r['I_A']:8.1f} {r['N']:6d} "
              f"{bc:14.6f} {br:14.6f} {bd:12.2e} "
              f"{r.get('b3_ppb', np.nan):10.4f} "
              f"{r.get('b3_maxdiff', np.nan):12.2e}")

print(f"\n{'='*80}")
print("Per-plateau tables complete. See plots in next cell.")

Seg 1: 3 plateaus identified (I levels: ['-0', '1740', '0'])
Seg 2: 3 plateaus identified (I levels: ['-0', '1740', '0'])
Seg 3: 3 plateaus identified (I levels: ['-0', '1740', '0'])
Seg 4: 3 plateaus identified (I levels: ['-0', '1740', '0'])
Seg 5: 3 plateaus identified (I levels: ['-0', '1740', '0'])

SEGMENT 1 -- PER-PLATEAU AVG PRECISION (ppb rel. main field)  [3 plateaus]
  #     I(A)      N      Bmain       b2       b3       b4       b5       b6       b7       b8       b9      b10      b11      b12      b13      b14      b15    worst
---------------------------------------------------------------------------------------------------------------------------------------------------------------------
  0      0.0   3738     0.0000   0.0000   0.0000   0.0000   0.0000   0.0000   0.0000   0.0000   0.0000   0.0000   0.0000   0.0000   0.0000   0.0000   0.0000   0.0000
  1   1740.0   3526    64.2135   0.9420   0.3520   1.4550   1.2759   2.1001   1.4158   0.0188   1.2450   1.6258   0.7474 

In [11]:
# ============================================================
# PER-PLATEAU PLOTS -- b3 FOCUS + PRECISION HEATMAPS
# ============================================================

colors_seg = plt.cm.Set1(np.linspace(0, 0.6, N_SEGMENTS))
hlabels = ['Bm'] + [f'b{h}' for h in range(2, 16)]

fig, axes = plt.subplots(4, 2, figsize=(18, 24))

# ---- (0,0) b3 value vs current (computed and reference) ----
ax = axes[0, 0]
for seg in range(1, N_SEGMENTS + 1):
    I = [r['I_A'] for r in all_prec[seg]]
    c = [r.get('b3_avg_c', np.nan) for r in all_prec[seg]]
    rv = [r.get('b3_avg_r', np.nan) for r in all_prec[seg]]
    ax.plot(I, c, 'o-', color=colors_seg[seg-1], ms=3, label=f'Seg{seg} comp')
    ax.plot(I, rv, 'x--', color=colors_seg[seg-1], ms=3, alpha=0.3)
ax.set_xlabel("I (A)"); ax.set_ylabel("b3 (Units)")
ax.set_title("b3 vs Current: Computed (o) vs Reference (x)")
ax.legend(fontsize=6, ncol=2); ax.grid(True, alpha=0.3)

# ---- (0,1) b3 precision (ppb) vs current ----
ax = axes[0, 1]
for seg in range(1, N_SEGMENTS + 1):
    I = [r['I_A'] for r in all_prec[seg]]
    ppb = [r.get('b3_ppb', np.nan) for r in all_prec[seg]]
    ax.semilogy(I, ppb, 'o-', color=colors_seg[seg-1], ms=4, label=f'Seg {seg}')
ax.axhline(1.0, color='red', ls='--', alpha=0.5, label='1 ppb')
ax.set_xlabel("I (A)"); ax.set_ylabel("b3 precision (ppb)")
ax.set_title("b3 Average Precision at Each Plateau")
ax.legend(fontsize=6); ax.grid(True, alpha=0.3)

# ---- (1,0) Heatmap Seg 3 -- normal harmonics (bn) ----
ax = axes[1, 0]
seg_hm = 3
rows3 = all_prec[seg_hm]
ppb_mat = np.full((len(rows3), 15), np.nan)
for i, r in enumerate(rows3):
    ppb_mat[i, 0] = r['Bm_ppb']
    for h in range(2, 16): ppb_mat[i, h-1] = r.get(f'b{h}_ppb', np.nan)
im = ax.imshow(np.log10(np.clip(ppb_mat, 1e-8, None)), aspect='auto',
               cmap='RdYlGn_r', interpolation='nearest')
ax.set_xticks(range(15)); ax.set_xticklabels(hlabels, fontsize=7, rotation=45)
ylbl = [f"#{r['plat']} ({r['I_A']:.0f}A)" for r in rows3]
ax.set_yticks(range(len(rows3))); ax.set_yticklabels(ylbl, fontsize=5)
ax.set_title(f"Seg {seg_hm}: Normal (bn) Precision Heatmap (log10 ppb)")
ax.set_xlabel("Harmonic"); ax.set_ylabel("Plateau")
plt.colorbar(im, ax=ax, label='log10(ppb)', shrink=0.8)

# ---- (1,1) Heatmap Seg 5 (worst end segment) ----
ax = axes[1, 1]
seg_hm5 = 5
rows5 = all_prec[seg_hm5]
ppb_mat5 = np.full((len(rows5), 15), np.nan)
for i, r in enumerate(rows5):
    ppb_mat5[i, 0] = r['Bm_ppb']
    for h in range(2, 16): ppb_mat5[i, h-1] = r.get(f'b{h}_ppb', np.nan)
im2 = ax.imshow(np.log10(np.clip(ppb_mat5, 1e-8, None)), aspect='auto',
                cmap='RdYlGn_r', interpolation='nearest')
ax.set_xticks(range(15)); ax.set_xticklabels(hlabels, fontsize=7, rotation=45)
ylbl5 = [f"#{r['plat']} ({r['I_A']:.0f}A)" for r in rows5]
ax.set_yticks(range(len(rows5))); ax.set_yticklabels(ylbl5, fontsize=5)
ax.set_title(f"Seg {seg_hm5} (end): Normal (bn) Precision Heatmap (log10 ppb)")
ax.set_xlabel("Harmonic"); ax.set_ylabel("Plateau")
plt.colorbar(im2, ax=ax, label='log10(ppb)', shrink=0.8)

# ---- (2,0) Worst ppb per harmonic -- all segs vs central only ----
ax = axes[2, 0]
worst_bn_all = np.zeros(15)
worst_bn_central = np.zeros(15)  # Seg 2,3,4 only
for seg in range(1, N_SEGMENTS + 1):
    for r in all_prec[seg]:
        if abs(r['I_A']) <= 10: continue  # skip 0A plateau
        for h in range(2, 16):
            v = r.get(f'b{h}_ppb', 0)
            if np.isfinite(v):
                worst_bn_all[h-1] = max(worst_bn_all[h-1], v)
                if seg in (2, 3, 4):
                    worst_bn_central[h-1] = max(worst_bn_central[h-1], v)
        worst_bn_all[0] = max(worst_bn_all[0], r['Bm_ppb'])
        if seg in (2, 3, 4):
            worst_bn_central[0] = max(worst_bn_central[0], r['Bm_ppb'])
x = np.arange(1, 16)
ax.bar(x - 0.15, worst_bn_all, 0.3, color='salmon', edgecolor='black', label='All 5 segs')
ax.bar(x + 0.15, worst_bn_central, 0.3, color='steelblue', edgecolor='black', label='Central (2,3,4)')
ax.axhline(1.0, color='red', ls='--', alpha=0.5, label='1 ppb')
ax.set_xlabel("Harmonic"); ax.set_ylabel("Worst avg ppb")
ax.set_title("Worst-case Precision: All Segs vs Central (|I|>10A)")
ax.set_xticks(x); ax.set_xticklabels(hlabels, fontsize=7, rotation=45)
ax.legend(fontsize=7); ax.grid(True, alpha=0.3, axis='y')

# ---- (2,1) b3 turn-by-turn diff per plateau (Seg 3) ----
ax = axes[2, 1]
c3 = full_arrays_cache[3]; nt3 = c3["n_turns"]
r3 = ref_data[3].iloc[:nt3]
if "b3(Units)" in r3.columns:
    db3 = c3["C_merged"][:nt3, 2].real - r3["b3(Units)"].values.astype(float)
    for j, p in enumerate(all_plat[3]):
        m = p['mask']
        ax.plot(np.where(m)[0], db3[m], '.', ms=0.3, alpha=0.2,
                label=f"{p['I_mean']:.0f}A" if j % 5 == 0 else None)
ax.set_xlabel("Turn"); ax.set_ylabel("b3 diff (Units)")
ax.set_title("Seg 3: b3 Turn-by-turn Diff at Each Plateau")
ax.legend(fontsize=5, ncol=3); ax.grid(True, alpha=0.3)

# ---- (3,0) Per-segment worst b3 ppb ----
ax = axes[3, 0]
w_b3 = [max((r.get('b3_ppb', 0) for r in all_prec[s]), default=0)
        for s in range(1, N_SEGMENTS + 1)]
bar_colors = ['orange' if s in (1, 5) else 'steelblue'
              for s in range(1, N_SEGMENTS + 1)]
ax.bar([f"Seg {s}" for s in range(1, N_SEGMENTS + 1)], w_b3,
       color=bar_colors, edgecolor='black')
ax.axhline(1.0, color='red', ls='--', alpha=0.5, label='1 ppb')
ax.set_ylabel("Worst b3 avg ppb")
ax.set_title("b3: Worst Plateau Precision per Segment\n(orange = end segs, blue = central)")
ax.legend(); ax.grid(True, alpha=0.3, axis='y')

# ---- (3,1) b3 max|turn diff| vs |current| ----
ax = axes[3, 1]
for seg in range(1, N_SEGMENTS + 1):
    Ia = [abs(r['I_A']) for r in all_prec[seg]]
    mx = [r.get('b3_maxdiff', np.nan) for r in all_prec[seg]]
    ax.semilogy(Ia, mx, 'o-', color=colors_seg[seg-1], ms=4, label=f'Seg {seg}')
ax.set_xlabel("|I| (A)"); ax.set_ylabel("max|b3 diff| per turn (Units)")
ax.set_title("b3 Worst Single-turn Diff vs |Current|")
ax.legend(fontsize=6); ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig("golden_standard_SM18_per_plateau.png", dpi=150, bbox_inches='tight')
plt.show()

# ============= PRECISION VERDICT =============
print("\n" + "=" * 80)
print("PER-PLATEAU PRECISION VERDICT")
print("=" * 80)

# Overall worst (all segments, |I| > 10A)
ow = 0
for seg in range(1, N_SEGMENTS + 1):
    for r in all_prec[seg]:
        if abs(r['I_A']) <= 10: continue
        for h in range(2, 16):
            v = r.get(f'b{h}_ppb', 0)
            if np.isfinite(v): ow = max(ow, v)
        ow = max(ow, r['Bm_ppb'])

# Central segments only (Seg 2, 3, 4)
ow_central = 0
for seg in (2, 3, 4):
    for r in all_prec[seg]:
        if abs(r['I_A']) <= 10: continue
        for h in range(2, 16):
            v = r.get(f'b{h}_ppb', 0)
            if np.isfinite(v): ow_central = max(ow_central, v)
        ow_central = max(ow_central, r['Bm_ppb'])

print(f"\n--- ALL 5 SEGMENTS (|I|>10A) ---")
print(f"  Worst avg precision: {ow:.4f} ppb")

print(f"\n--- CENTRAL SEGMENTS (2, 3, 4) ---")
print(f"  Worst avg precision: {ow_central:.4f} ppb")
if ow_central < 1.0:
    print(f"  VERDICT: ALL SUB-PPB -- PERFECT PARITY")
else:
    print(f"  VERDICT: {ow_central:.4f} ppb")

# Per-segment summary
print(f"\n--- PER-SEGMENT BREAKDOWN (1740A plateau, all harmonics) ---")
print(f"  {'Seg':>4} {'B_main(T)':>12} {'worst bn ppb':>14} {'worst bn':>10} {'b3 ppb':>10}")
print(f"  {'-'*55}")
for seg in range(1, N_SEGMENTS + 1):
    # Find worst bn at 1740A plateau
    for r in all_prec[seg]:
        if abs(r['I_A']) > 10:
            worst_h = max(((r.get(f'b{h}_ppb', 0), f'b{h}')
                          for h in range(2, 16) if np.isfinite(r.get(f'b{h}_ppb', 0))),
                         key=lambda x: x[0], default=(0, '-'))
            print(f"  {seg:4d} {r['Bm_avg_r']:12.4e} {worst_h[0]:14.4f} {worst_h[1]:>10s} "
                  f"{r.get('b3_ppb', 0):10.4f}")

# b3 specific
ow_b3 = max((r.get('b3_ppb', 0) for seg in range(1, N_SEGMENTS+1)
             for r in all_prec[seg] if abs(r['I_A']) > 10), default=0)
ow_b3_central = max((r.get('b3_ppb', 0) for seg in (2, 3, 4)
                     for r in all_prec[seg] if abs(r['I_A']) > 10), default=0)
print(f"\n--- b3 SUMMARY ---")
print(f"  b3 worst (all segs):    {ow_b3:.4f} ppb  (Seg 5, end segment)")
print(f"  b3 worst (central):     {ow_b3_central:.4f} ppb")
if ow_b3_central < 1.0:
    print(f"  b3 central: SUB-PPB")

# Per-harmonic worst (all segs)
print(f"\n--- PER-HARMONIC WORST PPB (all segs, |I|>10A) ---")
print(f"  {'Harmonic':>8} {'all segs':>12} {'central':>12}")
print(f"  {'-'*34}")
for idx, lbl in enumerate(hlabels):
    a = worst_bn_all[idx]
    c = worst_bn_central[idx]
    tag = '  <-- b3' if idx == 2 else ''
    sub = ' *' if c < 1.0 else ''
    print(f"  {lbl:>8s} {a:12.4f} {c:12.4f}{sub}{tag}")

print(f"\n  * = sub-ppb on central segments")

# End-segment explanation
print(f"\n--- NOTE ON END SEGMENTS (1, 5) ---")
for seg in (1, 5):
    for r in all_prec[seg]:
        if abs(r['I_A']) > 10:
            print(f"  Seg {seg}: B_main = {r['Bm_avg_r']:.4e} T at 1740A")
for seg in (3,):
    for r in all_prec[seg]:
        if abs(r['I_A']) > 10:
            print(f"  Seg {seg}: B_main = {r['Bm_avg_r']:.4e} T at 1740A (central)")
print(f"  End segments see ~1000x weaker main field -> ppb amplified")
print(f"  In absolute terms (Tesla), ALL differences are at machine precision")

print(f"\n{'='*80}")


PER-PLATEAU PRECISION VERDICT

--- ALL 5 SEGMENTS (|I|>10A) ---
  Worst avg precision: 68.5422 ppb

--- CENTRAL SEGMENTS (2, 3, 4) ---
  Worst avg precision: 46.1236 ppb
  VERDICT: 46.1236 ppb

--- PER-SEGMENT BREAKDOWN (1740A plateau, all harmonics) ---
   Seg    B_main(T)   worst bn ppb   worst bn     b3 ppb
  -------------------------------------------------------
     1  -5.0859e-03         2.4864        b15     0.3520
     2   1.7430e+00         0.2797         b7     0.0860
     3   2.9592e+00         0.0976         b9     0.0359
     4   1.7319e+00         0.5087         b2     0.1395
     5  -4.5151e-03         9.4463         b2     2.3315

--- b3 SUMMARY ---
  b3 worst (all segs):    2.3315 ppb  (Seg 5, end segment)
  b3 worst (central):     0.1395 ppb
  b3 central: SUB-PPB

--- PER-HARMONIC WORST PPB (all segs, |I|>10A) ---
  Harmonic     all segs      central
  ----------------------------------
        Bm      68.5422      46.1236
        b2       9.4463       0.5087 *
    

C:\Users\albellel\AppData\Local\Temp\ipykernel_23964\1625462758.py:131: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


---
## GUI Replication Instructions

To reproduce these results using the rotating-coil-analyzer GUI:

### Step 1: Load Data
1. Open the measurement folder:
   `golden_standards/golden_standard_SM18_01/HCMCBXFB012-E9000006_20241204_150437_stair_step_outer_negative/aperture1`
2. The GUI should discover 5 segment binary files: `*_corr_sigs_Ap_1_Seg{1..5}.bin`
3. Verify: **512 samples/turn**, shaft speed **60 RPM**

### Step 2: Coil Calibration (Kn)

**Option A -- Load from file** (recommended for exact parity):
- For each segment, load the corresponding Kn file:
  `*_Kn_values_Ap_1_Seg_{1..5}.txt`

**Option B -- Compute from head CSV**:
1. Load measurement head CSV: `CRMMMMH_AD-00000001.csv`
2. **Warm geometry: False** (cold geometry -- required for sub-ppb Kn parity)
3. Use design radius: **False** (use calibrated)
4. Number of multipoles: **15**
5. Connection formulas (from Parameters.txt):

   | Segment | abs | cmp |
   |---------|-----|-----|
   | 1 | `1.5` | `-1.3+1.5` |
   | 2 | `2.5` | `-2.3+2.5` |
   | 3 | `3.5` | `-3.3+3.5` |
   | 4 | `4.1` | `-4.3+4.1` |
   | 5 | `5.5` | `-5.3+5.5` |

### Step 3: Pipeline Configuration
1. **Magnet order**: **1** (dipole)
2. **Reference radius (Rref)**: **0.050 m** (50 mm)
3. **Absolute calibration**: **1.0**
4. **Pipeline options** (checkboxes):
   - [ ] dit -- **DISABLED** (binary files are `_corr_sigs_` = pre-corrected at hardware level)
   - [x] dri (drift + integrate)
   - [x] rot (rotation correction)
   - [x] cel (centre location)
   - [x] nor (normalisation)
   - [ ] fed (feeddown) -- disabled
5. **Drift mode**: **legacy**
6. **Merge mode**: **abs_upto_m_cmp_above** (absolute channel for harmonics up to main order, compensated above)
7. **legacy_rotate_excludes_last**: **True**

### Step 4: Verify Results
Compare against reference files: `*_results_Ap_1_Seg_{1..5}.txt` (tab-separated)

**Expected precision on current plateaus**:
- B_main(T): sub-ppb relative agreement on plateau averages
- b2..b15(Units): sub-ppb relative to main field on plateau averages
- Ramp turns (~0.6% of total) show larger differences -- this is expected

### Important Notes

> **Why dit is OFF**: The `_corr_sigs_` binary files have the di/dt correction
> pre-applied at the hardware/acquisition level. The `dit=true` flag in
> Parameters.txt refers to the original raw signal processing chain, not
> the pipeline step on these pre-corrected files. Applying dit again would
> double-correct and produce wrong results.
>
> **Why cold geometry**: The reference Kn files were generated with cold geometry
> scaling factors (0.997 for linear dimensions, 0.995 for magnetic surface).
> Using warm geometry (0.999999 scaling) produces ~0.5% Kn mismatch.

In [12]:
# ============================================================
# FINAL SUMMARY
# ============================================================
print("=" * 80)
print("GOLDEN STANDARD SM18 PARITY -- FINAL SUMMARY")
print("=" * 80)

print(f"\nDataset: HCMCBXFB012-E9000006 (dipole, stair-step outer-negative)")
print(f"Segments: {N_SEGMENTS}")
print(f"Turns per segment (ref): ~{ref_data[1].shape[0]:,}")
total_compared = sum(full_metrics[s].get("n_compare", ref_data[s].shape[0])
                     for s in range(1, N_SEGMENTS+1))
print(f"Total turns compared: {total_compared:,}")

print(f"\nBest configuration found by parameter sweep:")
print(f"  Options                    : {BEST_OPTIONS}")
print(f"  Drift mode                 : {BEST_DRIFT}")
print(f"  legacy_rotate_excludes_last: {BEST_ROT_EXCL}")
print(f"  Merge mode                 : {BEST_MERGE}")

# ALL-TURNS verdict
all_scores = [full_metrics[s]["total_score"] for s in range(1, N_SEGMENTS + 1)]
worst_all = max(all_scores)
print(f"\n--- ALL TURNS ---")
print(f"  Worst total score: {worst_all:.6e}")

# PLATEAU-ONLY verdict
plat_scores = [full_plat_metrics[s].get("total_score", float("inf"))
               for s in range(1, N_SEGMENTS + 1)]
worst_plat = max(plat_scores) if all(np.isfinite(s) for s in plat_scores) else float("inf")
print(f"\n--- PLATEAU TURNS ONLY ---")
print(f"  Worst total score: {worst_plat:.6e}")

print(f"\nPer-segment comparison:")
print(f"{'Seg':>4} {'ALL score':>14} {'PLAT score':>14} {'ALL Bmain':>14} "
      f"{'PLAT Bmain':>14}")
print("-" * 70)
for seg in range(1, N_SEGMENTS + 1):
    m = full_metrics[seg]
    pm = full_plat_metrics[seg]
    print(f"{seg:4d} {m['total_score']:14.6e} "
          f"{pm.get('total_score', float('nan')):14.6e} "
          f"{m['Bmain_maxabs']:14.6e} "
          f"{pm.get('Bmain_maxabs', float('nan')):14.6e}")

# Verdict
def _verdict(score):
    if score < 1e-8:
        return "EXCELLENT (machine precision)"
    elif score < 1e-3:
        return "GOOD (sub-milli-unit)"
    elif score < 1.0:
        return "CLOSE (sub-unit)"
    elif score < 100:
        return "MODERATE (tens of units)"
    else:
        return "MISMATCH (hundreds+ units)"

print(f"\n  ALL-TURNS verdict:     {_verdict(worst_all)}")
print(f"  PLATEAU-ONLY verdict:  {_verdict(worst_plat)}")
print("=" * 80)

# Clean up
del full_arrays_cache
gc.collect()
print("\nDone. Memory released.")

GOLDEN STANDARD SM18 PARITY -- FINAL SUMMARY

Dataset: HCMCBXFB012-E9000006 (dipole, stair-step outer-negative)
Segments: 5
Turns per segment (ref): ~57,019
Total turns compared: 285,095

Best configuration found by parameter sweep:
  Options                    : ('dri', 'rot', 'nor', 'cel')
  Drift mode                 : legacy
  legacy_rotate_excludes_last: True
  Merge mode                 : abs_upto_m_cmp_above

--- ALL TURNS ---
  Worst total score: 4.651943e+04

--- PLATEAU TURNS ONLY ---
  Worst total score: 1.074989e-01

Per-segment comparison:
 Seg      ALL score     PLAT score      ALL Bmain     PLAT Bmain
----------------------------------------------------------------------
   1   4.651943e+04   4.043843e-02   4.002155e-06   8.068388e-08
   2   1.285834e+02   2.764359e-03   2.852638e-04   1.308079e-05
   3   1.352194e+02   1.283496e-03   5.232194e-04   2.576333e-05
   4   1.126679e+02   6.155006e-03   2.811858e-04   1.281770e-05
   5   4.296976e+03   1.074989e-01   1.602954